# Test autoFRK

**Title**: Test autoFRK Functionality

**Author**: Hsu, Yao-Chih

**Reviewer**: Xie, Yi-Xuan

**Version**: 1141020

**Description**: This script tests the autoFRK python version in different scenarios.

**Reference**: Resolution Adaptive Fixed Rank Kringing by ShengLi Tzeng & Hsin-Cheng Huang

## Install our python autoFRK

In [1]:
import sys
import numpy as np
import torch
print("=" * 50)
print("Python Environment Info")
print("=" * 50)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")
print()

print("=" * 50)
print("Package Locations")
print("=" * 50)
print(f"Torch location: {torch.__file__}")
print(f"NumPy location: {np.__file__}")
print()

print("=" * 50)
print("PyTorch Info")
print("=" * 50)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA build: {torch.version.cuda}")
print()

print("=" * 50)
print("Test PyTorch Computation")
print("=" * 50)
x = torch.rand(3, 3)
print(f"Random tensor:\n{x}")
print(f"Sum: {x.sum().item():.4f}")

Python Environment Info
Python executable: d:\Github\autoFRK-python\test\.venv-gpu\Scripts\python.exe
Python version: 3.12.10

Package Locations
Torch location: d:\Github\autoFRK-python\test\.venv-gpu\Lib\site-packages\torch\__init__.py
NumPy location: d:\Github\autoFRK-python\test\.venv-gpu\Lib\site-packages\numpy\__init__.py

PyTorch Info
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA build: 12.4

Test PyTorch Computation
Random tensor:
tensor([[0.2841, 0.8341, 0.8912],
        [0.2298, 0.0240, 0.8658],
        [0.2147, 0.7006, 0.6625]])
Sum: 4.7068


In [2]:
import shutil
if shutil.which("dot") is None:
    error_msg = "Graphviz 'dot' executable not found. Please install Graphviz from https://graphviz.org/download/ and ensure it is added to your system PATH. Then restart your computer to apply the changes."
    raise EnvironmentError(error_msg)

In [3]:
# install autoFRK in development mode
import os
import sys
module_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

!"{sys.executable}" -m pip uninstall -y autoFRK
!"{sys.executable}" -m pip install --upgrade pip build setuptools wheel matplotlib pandas torchviz graphviz
!"{sys.executable}" -m build {module_root}
!"{sys.executable}" -m pip install -e "{module_root}"

Found existing installation: autoFRK 1.1.1
Uninstalling autoFRK-1.1.1:
  Successfully uninstalled autoFRK-1.1.1
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools>=61.0
  - wheel
* Getting build dependencies for sdist...
running egg_info
writing src\autoFRK.egg-info\PKG-INFO
writing dependency_links to src\autoFRK.egg-info\dependency_links.txt
writing requirements to src\autoFRK.egg-info\requires.txt
writing top-level names to src\autoFRK.egg-info\top_level.txt
reading manifest file 'src\autoFRK.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'src\autoFRK.egg-info\SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src\autoFRK.egg-info\PKG-INFO
writing dependency_links to src\autoFRK.egg-info\dependency_links.txt
writing requirements to src\autoFRK.egg-info\requires.txt
writing top-level names to src\autoFRK.egg-info\top_level.txt
reading manifes

C:\Users\Yi-Xuan\AppData\Local\Temp\build-env-9l70la8g\Lib\site-packages\setuptools\dist.py:483: SetuptoolsDeprecationWarning: Cannot find any files for the given pattern.
!!

        ********************************************************************************
        Pattern 'LICENCE*' did not match any files.

        By 2026-Mar-20, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.
        ********************************************************************************

!!
  for path in sorted(cls._find_pattern(pattern, enforce_match))
C:\Users\Yi-Xuan\AppData\Local\Temp\build-env-9l70la8g\Lib\site-packages\setuptools\dist.py:483: SetuptoolsDeprecationWarning: Cannot find any files for the given pattern.
!!

        ********************************************************************************
        Pattern 'LICENCE*' did not match any files.

        By 2026-Mar-20, you need to update your project and remove depr

Obtaining file:///D:/Github/autoFRK-python
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for autoFRK (pyproject.toml): started
  Building editable for autoFRK (pyproject.toml): finished with status 'done'
  Created wheel for autoFRK: filename=autofrk-1.1.1-0.editable-py3-none-any.whl size=17357 sha256=0f6a11a1500f0744b8dee93df6f8821f7dfe15d08835b9195c170ddbf6f13a8c
  Stored in directory: C:\Users\Yi-Xuan\AppData\Local\Temp\pip-ephem-wheel-cache-t_o3ideq\wheels\b5\c9\34\8b4aeb82bdd4605446fbf16719b76d90542e8e329626c5ab70
Succe

## Import modules

In [4]:
# import modules
import os
import sys
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from autoFRK import AutoFRK
from autoFRK.utils.utils import to_tensor, p

## Version

In [5]:
!pip show autoFRK

Name: autoFRK
Version: 1.1.1
Summary: autoFRK: Automatic Fixed Rank Kriging. The Python version with PyTorch
Home-page: https://github.com/Josh-test-lab/autoFRK-python
Author: ShengLi Tzeng, Hsin-Cheng Huang, Wen-Ting Wang, Yao-Chih Hsu
Author-email: 
License-Expression: GPL-3.0-or-later
Location: D:\Github\autoFRK-python\test\.venv-gpu\Lib\site-packages
Editable project location: D:\Github\autoFRK-python
Requires: colorlog, faiss-cpu, numpy, pandas, scikit-learn, scipy, torch
Required-by: 


In [6]:
from importlib.metadata import metadata

meta = metadata("autoFRK")
for key in meta:
    print(f"{key}: {meta[key]}")

Metadata-Version: 2.4
Name: autoFRK
Version: 1.1.1
Summary: autoFRK: Automatic Fixed Rank Kriging. The Python version with PyTorch
Author: ShengLi Tzeng, Hsin-Cheng Huang, Wen-Ting Wang, Yao-Chih Hsu
Maintainer: Yao-Chih Hsu
License-Expression: GPL-3.0-or-later
Project-URL: Homepage, https://github.com/Josh-test-lab/autoFRK-python
Project-URL: Homepage, https://github.com/Josh-test-lab/autoFRK-python
Keywords: kriging,spatial statistics,torch,autoFRK,geostatistics
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Classifier: Development Status :: 5 - Production/Stable
Requires-Python: >=3.10
Description-Content-Type: text/markdown
License-File: LICENSE
Requires-Dist: numpy>=1.23
Requires-Dist: numpy>=1.23

## Load data

In [7]:
# load data
datasets_path = f'test datasets/matrixForTest'
data = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_data.csv'))
locs = pd.read_csv(os.path.join(datasets_path, 'matrixForTest_locs.csv'))

In [8]:
data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-0.652166,-5.366158,3.143780,3.798089,-7.997662,1.136705,0.960326,3.201232,1.992290,2.001078,...,-0.448844,-0.577043,1.269652,0.347800,-3.742630,2.661689,-0.436828,-3.561380,-2.566957,3.640283
1,-0.816321,-6.481439,2.749028,0.729822,-4.332897,-2.829327,5.146832,-2.724882,3.281578,-0.628886,...,-3.152476,-0.342614,7.929967,-2.177937,0.556117,-2.043759,4.111130,-5.352733,1.529268,-1.917891
2,-4.549512,-7.763640,-1.872094,2.033659,-5.619806,-2.764594,6.608074,-1.398568,0.612283,-2.144521,...,-11.434087,3.868390,5.407404,7.577723,-6.269609,-1.125877,-13.487372,-0.227024,-2.769120,12.927128
3,-5.870963,-5.904798,-0.655968,7.860149,2.625713,-3.847751,7.439885,2.407433,3.273375,2.692631,...,-7.528925,6.847024,7.220447,4.332599,-3.998274,2.552754,-10.645833,3.684432,0.137623,7.370367
4,-6.552938,-8.789871,0.440326,6.932291,-0.442761,-4.145054,5.732851,0.655010,0.159350,-3.153300,...,-8.633517,4.540188,1.572350,7.350779,-3.499281,-0.169003,-10.041218,1.220481,-3.726887,8.765699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.790659,0.130805,5.131183,-2.309486,-4.404798,5.305249,1.197340,-0.427687,1.653997,4.622978,...,-0.952464,4.506945,-0.397343,-4.158340,0.220648,-1.642949,10.328322,-4.605859,1.894557,2.241329
96,1.685645,1.215962,3.793043,-3.899476,-1.595553,1.209884,-0.674625,-3.748975,-2.862852,1.977467,...,3.481229,0.753463,-4.839027,-5.032350,3.904153,6.617573,6.643477,3.399834,-1.562913,-2.786618
97,-1.395409,-3.042111,1.776225,-0.018072,-8.091349,-0.487496,-5.728769,0.954178,-1.206207,4.115169,...,-0.294438,1.971506,0.779557,-3.610582,2.661466,4.502825,2.828657,-1.532787,5.843775,3.382403
98,-4.765434,-4.096297,-6.376321,3.124180,10.963501,-6.188876,4.604844,4.612062,-2.310631,-7.859488,...,-4.794723,1.663460,3.688412,12.393074,-7.103671,-0.586811,-14.112674,6.708079,-2.698385,7.515730


In [9]:
locs

,Var1,Var2
0,0.310345,0.551724
1,0.517241,0.827586
2,0.413793,0.103448
3,0.517241,0.344828
4,0.689655,0.034483
...,...,...
95,0.275862,0.655172
96,0.344828,0.862069
97,0.379310,0.758621
98,0.620690,0.241379


## Convert data to tensor

In [10]:
# convert to tensor
data = to_tensor(data.to_numpy())
locs = to_tensor(locs.to_numpy())

## Random missing

In [11]:
import torch

def introduce_missing_data(data, p_miss=0.3):
    """
    Introduces missing values (NaN) into a 2D tensor.

    Ensures that no single row or single column is composed entirely of NaN values.

    Args:
        data (torch.Tensor): The input 2D tensor.
        p_miss (float): The approximate probability (between 0.0 and 1.0) 
                          of an element being missing.

    Returns:
        torch.Tensor: A new tensor with missing values.
    """
    
    # 1. Clone the data and ensure it's a float tensor (NaN is a float)
    if not torch.is_floating_point(data):
        data = data.float()
    data_missing = data.clone()
    
    # Get dimensions
    if data.dim() != 2:
        raise ValueError("This function is designed for 2D tensors.")
    rows, cols = data.shape

    # 2. Create a preliminary mask based on the missing probability
    # (True where data will be missing)
    mask = torch.rand(data.shape) < p_miss

    # 3. Check and fix any all-missing rows
    # torch.all(mask, dim=1) finds rows where all elements are True
    all_true_rows = torch.all(mask, dim=1)
    for row_idx in torch.where(all_true_rows)[0]:
        # If a row is all True, pick one random column in that row
        col_to_fix = torch.randint(0, cols, (1,))
        # And set its mask to False (it will NOT be missing)
        mask[row_idx, col_to_fix] = False

    # 4. Check and fix any all-missing columns
    # torch.all(mask, dim=0) finds columns where all elements are True
    all_true_cols = torch.all(mask, dim=0)
    for col_idx in torch.where(all_true_cols)[0]:
        # If a col is all True, pick one random row in that column
        row_to_fix = torch.randint(0, rows, (1,))
        # And set its mask to False (it will NOT be missing)
        mask[row_to_fix, col_idx] = False

    # 5. Apply the final, corrected mask
    data_missing[mask] = float('nan')
    
    return data_missing

# --- Example Usage ---

# 1. Create a sample tensor (e.g., 10x10 of integers)
my_data = torch.randint(0, 100, (10, 8))
print("--- Original Data ---")
print(my_data)

# 2. Introduce ~30% missing values
missing_data = introduce_missing_data(my_data, p_miss=0.3)
print("\n--- Data With Missing Values (NaN) ---")
print(missing_data)

# 3. Verification (check if any row or col is ALL NaN)
row_all_nan = torch.all(missing_data.isnan(), dim=1).any()
col_all_nan = torch.all(missing_data.isnan(), dim=0).any()

print(f"\nAny entire row missing?  {row_all_nan}")
print(f"Any entire col missing?  {col_all_nan}")

--- Original Data ---
tensor([[21, 54,  6, 98, 94,  6, 50, 66],
        [11, 13, 21, 60,  7,  4, 75, 98],
        [ 0, 68, 40, 61, 22, 88, 65, 67],
        [21, 97, 78, 13,  3, 62, 10, 49],
        [63, 95, 76, 73, 60, 53, 49, 29],
        [37, 42,  6, 84, 78, 21, 42, 72],
        [57, 27, 25, 15, 22, 39, 20, 98],
        [27, 44,  0, 88, 57, 38, 82, 41],
        [91, 46, 54, 60, 34, 44, 22, 69],
        [20,  7, 25,  0, 25, 51, 39, 78]])

--- Data With Missing Values (NaN) ---
tensor([[21., nan,  6., 98., nan, nan, 50., 66.],
        [11., 13., 21., 60.,  7., nan, 75., 98.],
        [nan, 68., 40., 61., nan, 88., 65., nan],
        [nan, nan, 78., 13.,  3., 62., 10., 49.],
        [nan, nan, 76., 73., 60., 53., 49., 29.],
        [37., 42.,  6., 84., 78., nan, nan, nan],
        [nan, 27., 25., nan, 22., 39., 20., 98.],
        [nan, nan,  0., nan, nan, 38., 82., 41.],
        [91., nan, nan, 60., 34., 44., 22., 69.],
        [nan, nan, nan,  0., 25., 51., 39., 78.]])

Any entire row 

In [12]:
data = introduce_missing_data(data, p_miss=0.3)
print(data)

tensor([[-0.6522,     nan,     nan,  ..., -3.5614, -2.5670,     nan],
        [    nan, -6.4814,  2.7490,  ..., -5.3527,  1.5293, -1.9179],
        [    nan, -7.7636, -1.8721,  ..., -0.2270, -2.7691, 12.9271],
        ...,
        [    nan, -3.0421,  1.7762,  ..., -1.5328,  5.8438,     nan],
        [-4.7654,     nan,     nan,  ...,     nan, -2.6984,     nan],
        [ 3.1509,     nan,  1.9028,  ...,     nan, -0.4252,     nan]],
       dtype=torch.float64)


## Test on known locations

In [13]:
model = AutoFRK()
model.forward(
    data=data,
    loc=locs,
    method='EM',
    requires_grad=True,
    device='cpu'
)

2025-10-22 19:20:15 - autoFRK.utils.logger - WARNING: Parameter "device" was not set. Value "cuda:0" detected and used.
2025-10-22 19:20:15 - autoFRK.utils.logger - INFO: Successfully using device "cpu".
2025-10-22 19:20:15 - autoFRK.utils.logger - INFO: Gradient tracking has been enabled for autoFRK.
2025-10-22 19:20:15 - autoFRK.utils.logger - INFO: Calculate TPS with rectangular coordinates.
2025-10-22 19:20:21 - autoFRK.utils.logger - INFO: Number of iteration: 23


{'M': tensor([[ 7.1492e+00,  5.2764e+00, -4.3007e+00, -4.3204e+00, -1.1626e+00,
          -3.6482e-01,  4.5813e-01,  4.9105e-01, -1.9224e-01,  4.9482e-01,
           1.4810e-01, -1.3373e-01],
         [ 5.2764e+00,  3.9217e+00, -3.2075e+00, -3.1280e+00, -8.1561e-01,
          -2.3776e-01,  3.6493e-01,  4.0967e-01, -1.3213e-01,  4.4844e-01,
           1.5501e-01, -4.8554e-02],
         [-4.3007e+00, -3.2075e+00,  2.6759e+00,  2.4096e+00,  5.5225e-01,
           1.2798e-01, -3.4904e-01, -4.4107e-01,  1.1164e-01, -5.2047e-01,
          -2.0768e-01, -1.4933e-02],
         [-4.3204e+00, -3.1280e+00,  2.4096e+00,  3.0441e+00,  1.0505e+00,
           4.3516e-01, -1.1482e-01,  4.4247e-02,  1.1166e-01,  2.0362e-01,
           1.7546e-01,  2.8626e-01],
         [-1.1626e+00, -8.1561e-01,  5.5225e-01,  1.0505e+00,  4.7380e-01,
           2.3400e-01,  5.2834e-02,  1.9729e-01,  2.1371e-02,  3.1868e-01,
           1.8757e-01,  1.7926e-01],
         [-3.6482e-01, -2.3776e-01,  1.2798e-01,  4.3516e-01

In [14]:
pred = model.predict()

### Check gradient tracking

In [15]:
from torchviz import make_dot
SAVE_DIR = "gradient tracking/all known locations"
os.makedirs(SAVE_DIR, exist_ok=True)

for k, v in model.obj.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

for k, v in model.obj['G'].items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

for k, v in pred.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

M: requires_grad=True;  grad_fn=<DivBackward0 object at 0x00000237D65FC1C0>; is_leaf=False
s: requires_grad=True;  grad_fn=<MaximumBackward0 object at 0x00000237D779C220>; is_leaf=False
w: requires_grad=True;  grad_fn=<CopySlices object at 0x00000237D779C220>; is_leaf=False
V: requires_grad=True;  grad_fn=<SubBackward0 object at 0x00000237D779C220>; is_leaf=False
MRTS: requires_grad=False;  grad_fn=None; is_leaf=True
UZ: requires_grad=False;  grad_fn=None; is_leaf=True
Xu: requires_grad=False;  grad_fn=None; is_leaf=True
nconst: requires_grad=False;  grad_fn=None; is_leaf=True
BBBH: requires_grad=False;  grad_fn=None; is_leaf=True
pred.value: requires_grad=True;  grad_fn=<AddBackward0 object at 0x00000237D779C220>; is_leaf=False


In [16]:
import torch.nn.functional as F

F.mse_loss(pred['pred.value'].cpu(), data.cpu()).item()

nan

```r
> options(digits = 15)
> print(mse)
[1] 11.8539587159961 # R mse result
```

In [17]:
# for i in range(data.shape[1]):
#     y_pred = pred['pred.value'][:, i].detach().cpu()
#     y_true = data[:, i].detach().cpu()

#     tmp = F.mse_loss(y_pred, y_true)
#     print(tmp)

#     plt.figure(figsize=(8,5))
#     plt.plot(y_true, label='True Value', marker='o')
#     plt.plot(y_pred, label='Predicted Value', marker='x')
#     plt.xlabel('Sample Index')
#     plt.ylabel('Value')
#     plt.title(f'Prediction vs True Value at {i} Step with error {round(tmp.item(), 2)}')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

In [18]:
for i in range(data.shape[1]):
    y_pred = pred['pred.value'][:, i].cpu()
    y_true = data[:, i].cpu()

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


### Compare with r result

```r
> temp # r residual result
 [1]  7.9766306122139898 12.0116817669945561  8.0081251964859330  3.5373058520311229 13.8528751726772583
 [6]  1.7858863206045086  9.8859536589469261 30.0424029115677840 27.0729660181976612  0.2159766252862333
[11]  0.0752655442993194  5.8887993596240475 12.5987205947255649 11.4932438517303979 37.7533887912337889
[16]  1.7491903038108714  4.2473651696074111  6.6476433759056297  4.3364578111025098  5.5149082237930296
[21] 23.4936795233438822  5.3612826806645080 10.3639904061309522 22.0650623239508086  3.2049127751009370
[26]  5.1869105672897371 30.8554320276935812  1.3647053212213505  9.0990348107428840 39.9289638829070839
```

經過確認，R 與 Python 的結果是大致一致的。
全時間點與各時間點的 MSE ，與 R 的結果皆相差不大，精準度介於小數點後6位元至10位元。

## Test on unknown locations
### Train:Test = 7:3

In [19]:
## split data into train and test
train_data = data[:70, :]
test_data = data[70:, :]
train_locs = locs[:70, :]
test_locs = locs[70:, :]

In [20]:
## training on train data
model = AutoFRK()
model.forward(
    data=train_data,
    loc=train_locs,
    method='EM',
    requires_grad=True
)

## predict on test data
pred = model.predict(
    newloc = test_locs
)

F.mse_loss(pred['pred.value'].cpu(), test_data.cpu()).item()


2025-10-22 19:20:21 - autoFRK.utils.logger - WARNING: Parameter "device" was not set. Value "cuda:0" detected and used.
2025-10-22 19:20:21 - autoFRK.utils.logger - INFO: Gradient tracking has been enabled for autoFRK.
2025-10-22 19:20:21 - autoFRK.utils.logger - INFO: Calculate TPS with rectangular coordinates.


KeyboardInterrupt: 

In [ ]:

for i in range(test_data.shape[1]):
    y_pred = pred['pred.value'][:, i].cpu()
    y_true = test_data[:, i].cpu()

    tmp = F.mse_loss(y_pred, y_true)
    print(tmp.item())

In [ ]:
model.obj["G"]["BBBH"].shape

In [ ]:
p(model.obj)

### Compare with R result

**R MSPE(ALL TIME): 18.8886195905131**

**PYTHON MSPE(ALL TIME): 18.88861955720493**

| Time | R MSPE | Python MSPE | Difference |
|------|--------|-------------|------------|
| 1 | 11.4394091240148 | 11.439409120905758 | 0.000000003108942 |
| 2 | 25.1077785518079 | 25.107778518456907 | 0.000000033350993 |
| 3 | 27.4500733520329 | 27.450073356685994 | -0.000000004653094 |
| 4 | 11.8248343539311 | 11.824834303775908 | 0.000000050155192 |
| 5 | 14.6350201943092 | 14.635020084480068 | 0.000000109829132 |
| 6 | 9.47801965023193 | 9.47801959933821 | 0.00000005089372 |
| 7 | 13.7277971543547 | 13.727797185025869 | -0.000000030671169 |
| 8 | 12.1724361254889 | 12.172436124250249 | 0.000000001238651 |
| 9 | 17.8483094625563 | 17.848309462917722 | -0.000000000361522 |
| 10 | 11.4141953861842 | 11.414195386232246 | -0.000000000047954 |
| 11 | 16.1900735681208 | 16.1900735700657 | -0.000000001944900 |
| 12 | 12.385319820021 | 12.385319820401966 | -0.000000000380966 |
| 13 | 10.4332040960123 | 10.43320407661182 | 0.00000001940048 |
| 14 | 41.7785075156215 | 41.7785074506677 | 0.0000000649538 |
| 15 | 38.0121380265941 | 38.01213778080117 | 0.00000024579293 |
| 16 | 27.1439462145094 | 27.14394624582416 | -0.00000003131476 |
| 17 | 9.50105793733591 | 9.501057920471283 | 0.000000016864627 |
| 18 | 18.9383195599305 | 18.93831947016505 | 0.00000008976545 |
| 19 | 17.3566153448881 | 17.35661536394406 | -0.00000001905596 |
| 20 | 19.450701347322 | 19.450701334660998 | 0.000000012661002 |
| 21 | 15.1800288763207 | 15.180028847571453 | 0.000000028749247 |
| 22 | 17.5135203624483 | 17.513520315329163 | 0.000000047119137 |
| 23 | 15.1051590271628 | 15.105158902227114 | 0.000000124935686 |
| 24 | 35.5388334703914 | 35.538833666779524 | -0.000000196388124 |
| 25 | 21.3261982841632 | 21.32619837250057 | -0.00000008833737 |
| 26 | 9.027353231579 | 9.027353232868238 | -0.000000001289238 |
| 27 | 30.1307246819617 | 30.130724681830557 | 0.000000000131143 |
| 28 | 19.6400677244916 | 19.640067727176127 | -0.000000002684527 |
| 29 | 12.2678522027652 | 12.267852189968524 | 0.000000012796676 |
| 30 | 24.6410930688427 | 24.641092604213856 | 0.000000464628844 |

**結論**：R 與 Python 的結果高度一致，差異範圍在 10^-7 到 10^-10 之間，證明 Python 實現的正確性。

### Check gradient tracking

In [ ]:
from torchviz import make_dot
SAVE_DIR = "gradient tracking/with unknown locations"
os.makedirs(SAVE_DIR, exist_ok=True)

for k, v in model.obj.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

for k, v in model.obj['G'].items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

for k, v in pred.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: requires_grad={v.requires_grad};  grad_fn={v.grad_fn}; is_leaf={v.is_leaf}")
        # file_path = os.path.join(SAVE_DIR, k)
        # dot = make_dot(v)
        # dot.attr(dpi='300')
        # dot.render(file_path, format='png')

In [ ]:
# for i in range(test_data.shape[1]):
#     y_pred = pred['pred.value'][:, i].detach().cpu()
#     y_true = test_data[:, i].detach().cpu()

#     tmp = F.mse_loss(y_pred, y_true)
#     print(tmp.item())

#     plt.figure(figsize=(8,5))
#     plt.plot(y_true, label='True Value', marker='o')
#     plt.plot(y_pred, label='Predicted Value', marker='x')
#     plt.xlabel('Sample Index')
#     plt.ylabel('Value')
#     plt.title(f'Prediction vs True Value at {i} Step with error {round(tmp.item(), 2)}')
#     plt.legend()
#     plt.grid(True)
#     plt.show()